In [1]:
import nltk
import pymorphy2

import ujson as json
import matplotlib.pyplot as plt
import numpy as np
import itertools
import gzip

from datetime import datetime

from collections import Counter

In [2]:
class Document:
    def __init__(self, init_dict):
        self.title = init_dict.get('title', '')
        self.description = init_dict.get('description', '')
        self.url = init_dict.get('url', '')
        self.site = init_dict.get('site', '')
        self.ts = datetime.fromtimestamp(init_dict['ts']) if 'ts' in init_dict else -1
    
    def __str__(self):
        res = ''
        res += 'url : %s\n' % self.url
        res += 'date : %s\n' % self.ts
        res += 'title : %s\n' % self.title
        res += 'description : %s\n' % self.description
        res += 'site : %s\n' % self.site
        return res

In [3]:
fin = gzip.open('dataset_mai.jsonl.gz')
for line in itertools.islice(fin, 10):
    data = json.loads(line.strip())
    print(Document(data))

url : http://bloknot-volzhsky.ru/news/volzhane-mogut-podat-zayavlenie-na-letnie-putevki-
date : 2019-11-30 18:26:10
title : Волжане могут подать заявление на летние путевки для детей
description : С понедельника заявления начинают принимать в МФЦ
site : bloknot-volzhsky.ru

url : https://trikky.ru/test-na-znanie-russkogo-yazyka-423354.html
date : 2019-11-30 18:26:48
title : 💗Тест на знание русского языка💗
description : Тест со сложными и легкими вопросами. Для кого-то будет легко набрать все 100 баллов, а кому-то будет немного тяжело. В любом случае попробовать стоит.1. Что изучает фразеология? способы образования слов устойчивые сочетания слов части речи2. На месте каких цифр в словах пишется н? В простенке между занавеше(1)ыми окнами были установле(2)ы часы, а рядом с ними […]
site : trikky.ru

url : https://topwar.ru/165315-chernomorskij-flot-popolnilsja-150-tonnym-plavkranom-proekta-02690.html
date : 2019-11-30 18:26:43
title : Черноморский флот пополнился 150-тонным плавкраном про

In [4]:
fin = gzip.open('dataset_mai.jsonl.gz')
dataset = []
dataset_test = []
for line in itertools.islice(fin, 10000):
    data = json.loads(line.strip())
    dataset.append(Document(data))
    
for line in itertools.islice(fin, 5000):
    data = json.loads(line.strip())
    dataset_test.append(Document(data))

In [5]:
test_doc = dataset[11]
print(test_doc)

url : https://trikky.ru/kakoy-ty-napitok-423256.html
date : 2019-11-30 18:26:48
title : Какой ты напиток?
description : Какой ты напиток?Зелёненький,свеженький мохито,или взрывной лимонад?А может быть,Кока Кола?Сейчас  и узнаем!1. Какой вкус в напитках тебе больше всего нравится? Яркие цвета - вот что самое главное! Эта многослойность напитков заставляет завораживаться... Сладость,я такая сладкоежка! Я обожаю газировку,такая взрывная,как шипучка!2. Какие типы цветов напитков тебе больше нравятся? Холодные(голубые,синие,фиолетовые разных оттенков) Чёрный/Белый/Белый в крапинку Зелёные,цвет природы. Яркие […]
site : trikky.ru



### Как поделить текст на слова?

In [6]:
def print_words(a_document, a_split_func):
    print('------TITLE----------')
    for word in a_split_func(test_doc.title):
        print(word)

    print('------DESCRIPTION----------')
    for word in a_split_func(test_doc.description):
        print(word)

In [7]:
### Побить по пробелам?
def split_words_v1(a_text):
    for word in a_text.split():
        yield word

print_words(test_doc, split_words_v1)

------TITLE----------
Какой
ты
напиток?
------DESCRIPTION----------
Какой
ты
напиток?Зелёненький,свеженький
мохито,или
взрывной
лимонад?А
может
быть,Кока
Кола?Сейчас
и
узнаем!1.
Какой
вкус
в
напитках
тебе
больше
всего
нравится?
Яркие
цвета
-
вот
что
самое
главное!
Эта
многослойность
напитков
заставляет
завораживаться...
Сладость,я
такая
сладкоежка!
Я
обожаю
газировку,такая
взрывная,как
шипучка!2.
Какие
типы
цветов
напитков
тебе
больше
нравятся?
Холодные(голубые,синие,фиолетовые
разных
оттенков)
Чёрный/Белый/Белый
в
крапинку
Зелёные,цвет
природы.
Яркие
[…]


In [8]:
### Побить по всем не буквам и цифрам?
def split_words_v2(a_text):
    cur_word = ''
    for letter in a_text:
        if letter.isalnum():
            cur_word += letter
        else:
            if cur_word: yield cur_word
            cur_word = ''
            
    if cur_word: yield cur_word
    
print_words(test_doc, split_words_v2)

------TITLE----------
Какой
ты
напиток
------DESCRIPTION----------
Какой
ты
напиток
Зелёненький
свеженький
мохито
или
взрывной
лимонад
А
может
быть
Кока
Кола
Сейчас
и
узнаем
1
Какой
вкус
в
напитках
тебе
больше
всего
нравится
Яркие
цвета
вот
что
самое
главное
Эта
многослойность
напитков
заставляет
завораживаться
Сладость
я
такая
сладкоежка
Я
обожаю
газировку
такая
взрывная
как
шипучка
2
Какие
типы
цветов
напитков
тебе
больше
нравятся
Холодные
голубые
синие
фиолетовые
разных
оттенков
Чёрный
Белый
Белый
в
крапинку
Зелёные
цвет
природы
Яркие


In [9]:
### Что делать с разными написаниями iphone6 и iphone 6 ? Как сделать чтобы всегда получать одну версию разбиения?
## Можно оставить студентам написать?
test_text = 'Где купить iPhone 6? и почему iPhone6 стоит так дорого?'
print(list(split_words_v1(test_text)))
print(list(split_words_v2(test_text)))

['Где', 'купить', 'iPhone', '6?', 'и', 'почему', 'iPhone6', 'стоит', 'так', 'дорого?']
['Где', 'купить', 'iPhone', '6', 'и', 'почему', 'iPhone6', 'стоит', 'так', 'дорого']


## Реализовать функцию которая единообразно будет разбивать такие строки?

In [10]:
def split_words_v3(a_text):
    cur_word = ''
    prev_is_alpha = False

    for letter in a_text:
        if  (letter.isalpha() and prev_is_alpha or 
            letter.isdigit() and not prev_is_alpha):
            cur_word += letter
        elif (letter.isalpha() and not prev_is_alpha or
             letter.isdigit() and prev_is_alpha):
            if cur_word: yield cur_word
            cur_word = letter
            prev_is_alpha = not prev_is_alpha
        else:
            if cur_word: yield cur_word
            cur_word = ''
            prev_is_alpha = False
    if cur_word: yield cur_word
         
print(list(split_words_v3(test_text)))

['Где', 'купить', 'iPhone', '6', 'и', 'почему', 'iPhone', '6', 'стоит', 'так', 'дорого']


In [11]:
words = list(split_words_v2(test_doc.title + ' ' + test_doc.description))

In [12]:
len(words)

74

In [13]:
uniq_words = Counter(words)
print("NumKeys : ", len(uniq_words))
print(uniq_words.most_common(10))

NumKeys :  63
[('Какой', 3), ('ты', 2), ('напиток', 2), ('в', 2), ('тебе', 2), ('больше', 2), ('Яркие', 2), ('напитков', 2), ('такая', 2), ('Белый', 2)]


## Могут ли быть разные написания одного слова?

In [14]:
def get_norm_word_v1(a_word):
    return a_word.lower()

uniq_words = Counter(get_norm_word_v1(word) for word in words)
print("NumKeys : ", len(uniq_words))
print(uniq_words.most_common(10))
### ('я', 2)

NumKeys :  62
[('какой', 3), ('ты', 2), ('напиток', 2), ('в', 2), ('тебе', 2), ('больше', 2), ('яркие', 2), ('напитков', 2), ('я', 2), ('такая', 2)]


In [15]:
from nltk.stem.snowball import RussianStemmer

In [16]:
stemmer = RussianStemmer()

In [17]:
stemmer.stem('Напитков')

'напитк'

In [18]:
def get_norm_word_v2(a_word):
    return stemmer.stem(a_word)

In [19]:
uniq_words = Counter(get_norm_word_v2(word) for word in words)
print("NumKeys : ", len(uniq_words))
print(uniq_words.most_common(10))
### ('я', 2)

NumKeys :  55
[('как', 5), ('напитк', 3), ('цвет', 3), ('ты', 2), ('напиток', 2), ('взрывн', 2), ('в', 2), ('теб', 2), ('больш', 2), ('нрав', 2)]


In [20]:
for word in words:
    if get_norm_word_v2(word) == 'как':
        print(word)

Какой
Какой
Какой
как
Какие


In [21]:
for word in words:
    if get_norm_word_v2(word) == 'цвет':
        print(word)

цвета
цветов
цвет


In [22]:
morth_analyzer = pymorphy2.MorphAnalyzer()

In [23]:
res = morth_analyzer.parse('Какая') #Какая , Напитков

In [24]:
res

[Parse(word='какая', tag=OpencorporaTag('ADJF,Apro femn,sing,nomn'), normal_form='какой', score=0.987012, methods_stack=((<DictionaryAnalyzer>, 'какая', 1676, 7),)),
 Parse(word='какая', tag=OpencorporaTag('GRND,impf,intr pres'), normal_form='какать', score=0.012987, methods_stack=((<DictionaryAnalyzer>, 'какая', 15, 67),))]

In [25]:
res[0].normal_form

'какой'

In [26]:
MORTH_CACHE = {}
def get_norm_word_v3(a_word):
    if a_word not in MORTH_CACHE: MORTH_CACHE[a_word] = morth_analyzer.parse(a_word)[0].normal_form
    return MORTH_CACHE[a_word]

In [27]:
uniq_words = Counter(get_norm_word_v3(word) for word in words)
print("NumKeys : ", len(uniq_words))
print(uniq_words.most_common(10))
### ('напиток', 2)

NumKeys :  56
[('напиток', 5), ('какой', 4), ('ты', 4), ('взрывной', 2), ('в', 2), ('большой', 2), ('нравиться', 2), ('яркий', 2), ('я', 2), ('такой', 2)]


In [28]:
### v4 - числа преоразовывать к специальному формату YYYY , DD - чтобы уменьшить словарь и склеивались похожие названия отличающиеся только цифрами 

### Пример деревьев решений для предсказания сайта по tf-idf векторам
Получится ли обучить хорошее решение линейной моделью?

In [29]:
def get_doc_words(a_doc, a_split=split_words_v3, a_norm_word=get_norm_word_v3):
    for word in itertools.chain(a_split(a_doc.title), a_split(a_doc.description)):
        yield a_norm_word(word)

In [30]:
class NaiveBayes:
    
    min_class_c = 12
    
    def __init__(self):
        self.word_in_class = Counter()
        self.word_not_in_class = Counter()
        
    def train(self, a_words, a_is_class):
        word_counter = self.word_in_class if a_is_class else self.word_not_in_class
        word_counter.update(a_words)
        
    def predict(self, a_words):
        res = 0
        num_words = 0
        for word in a_words:
            class_freq, not_class_freq = self.word_in_class.get(word, 0), self.word_not_in_class.get(word, 0)
            if class_freq < self.min_class_c or not_class_freq < self.min_class_c: continue
            num_words += 1
            res += np.log(class_freq/not_class_freq)
        return res/num_words if num_words > 0 else 0.0

In [31]:
dataset_sites = Counter(doc.site for doc in dataset)

sport_sites = set(['matchtv.ru', 'euro-football.ru', 'euro.com.ua',
                   'bobsoccer.ru', 'bombardir.ru', 'championat.com',
                   'football24.ru', 'rusfootball.info', 'soccernews.ru'
                  ])

for item in dataset_sites:
    if 'sport' in item: sport_sites.add(item)

In [32]:
for item in dataset_sites.most_common(1000):
    if item[0] not in sport_sites:
        print(item)

('tass.ru', 178)
('vk.com', 163)
('123ru.net', 143)
('gazeta.ru', 136)
('rossaprimavera.ru', 130)
('russian.rt.com', 118)
('daily-inform.ru', 108)
('newsyou.info', 95)
('365news.biz', 95)
('obozrevatel.com', 92)
('tvzvezda.ru', 83)
('moika78.ru', 78)
('life.ru', 73)
('wi-fi.ru', 71)
('riafan.ru', 71)
('rueconomics.ru', 69)
('pikabu.ru', 65)
('novosti-n.org', 63)
('mob-mobile.ru', 63)
('sm-news.ru', 58)
('news.ru', 58)
('mir24.tv', 57)
('nation-news.ru', 57)
('govoritmoskva.ru', 56)
('360tv.ru', 56)
('news-front.info', 53)
('terrikon.com', 53)
('vz.ru', 51)
('aif.ru', 51)
('dizainvfoto.ru', 50)
('argumenti.ru', 49)
('peterburg2.ru', 48)
('ukraina.ru', 46)
('oane.ws', 46)
('rosbalt.ru', 44)
('news.rambler.ru', 43)
('zelenj.ru', 40)
('ferra.ru', 39)
('bmwclub.ru', 39)
('habr.com', 39)
('deita.ru', 38)
('vestikavkaza.ru', 37)
('abnews.ru', 36)
('1rre.ru', 35)
('gazeta.spb.ru', 34)
('profile.ru', 34)
('fontanka.ru', 34)
('yerkramas.org', 33)
('riamo.ru', 33)
('radiosputnik.ria.ru', 32)
('ra

In [33]:
sport_dataset = [item for item in dataset if item.site in sport_sites]
not_sport_dataset = [item for item in dataset if item.site not in sport_sites]
print(len(sport_dataset), len(not_sport_dataset))

1087 8913


In [34]:
%%time

bayes_classifier = NaiveBayes()
for item in sport_dataset:
    bayes_classifier.train(get_doc_words(item), a_is_class=True)
    
for item in not_sport_dataset:
    bayes_classifier.train(get_doc_words(item), a_is_class=False)

Wall time: 22.7 s


In [35]:
for item in not_sport_dataset:
    predict = bayes_classifier.predict(get_doc_words(item))
    if predict > np.log(len(sport_dataset)/len(not_sport_dataset)):
        print(predict, item)
#         break
        print('-*-'*13)


-1.9601938853085725 url : https://rusturkey.com/post/207586/grazhdanka-rossii-vypala-s-balkona-7-go-etazha-v-antale
date : 2019-11-30 18:26:11
title : Гражданка России выпала с балкона 7-го этажа в Анталье
description : Жизнь женщины не удалось спасти
site : rusturkey.com

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.837474331018305 url : https://riavrn.ru/news/voronezhskiy-batutist-stal-prizerom-chempionata-mira/
date : 2019-11-30 18:26:38
title : Воронежский батутист стал призером чемпионата мира
description : Сергей Азарян отличился в паре с Михаилом Мельником из Тольятти.
site : riavrn.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-2.0732243592395294 url : https://tengrinews.kz/events/krupnyiy-pojar-ohvatil-bannyiy-kompleks-u-podnojiya-kok-tobe-385419/
date : 2019-11-30 18:22:16
title : Крупный пожар охватил банный комплекс у подножия Кок-Тобе
description : Крупный пожар охватил банный комплекс "Царское село", расположенный у подножия горы Кок-Тобе в Алматы вечером в субботу, 30 ноября

-1.920274681516339 url : https://gazeta.a42.ru/lenta/news/68406_stali-izvestny-gonorary-emelyanenko-i-koklyaeva-za-bokserski
date : 2019-11-30 19:02:24
title : Стали известны гонорары Емельяненко и Кокляева за боксерский поединок
description : Миллион рублей
site : gazeta.a42.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.8918603515477135 url : https://pravda-nn.ru/news/nizhegorodskie-sportsmeny-vyigrali-po-tri-zolotye-medali-na-kubke-rossii-po-sambo/
date : 2019-11-30 17:26:43
title : Нижегородские спортсмены выиграли по три золотые медали на Кубке России по самбо
description : В нижегородском театре оперы и балета каждый из регионов объявлял своих фаворитов. В зале собрались члены жюри и деятели в сфере культуры.
site : pravda-nn.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-0.8790297753911065 url : https://overclockers.ru/softnews/show/100556/novye-skrinshoty-fanatskogo-remejka-half-life-opposing-force
date : 2019-11-30 19:02:21
title : Новые скриншоты фанатского ремейка Half-Life: O

-2.013879266339763 url : https://bazaar.ru/heroes/parties/tatyana-navka-andrey-malahov-i-yuliya-hlynina-na-otkrytii-katka-v-gume/
date : 2019-11-30 19:36:10
title : Татьяна Навка, Андрей Малахов и Юлия Хлынина на открытии катка в ГУМе
description : Звездные гости встали на коньки
site : bazaar.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.6197618829187614 url : http://www.vestikavkaza.ru/material/283062
date : 2019-11-30 19:32:39
title : Рахим Чахкиев вызвал на бой Александра Емельяненко
description : Олимпийский чемпион по боксу 2008 года Рахим Чахкиев вызвал сегодня на бой бойца MMA Александра Емельяненко после того, как тот, победив пауэрлифтера Михаила Кокляева, заявил, что готов боксировать и с другими желающими бросить ему вызов …
site : vestikavkaza.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.8736712613144497 url : https://www.f1news.ru/news/f1-141436.html
date : 2019-11-30 19:36:43
title : Марио Изола: В гонке многое решит стратегия
description : Быстрейшей в завтрашней гон

-1.932235997565773 url : https://360tv.ru/news/obschestvo/rosatom-rassekretil-dokumenty-o-rozygryshah-kurchatova/
date : 2019-11-30 17:16:39
title : Росатом рассекретил документы о розыгрышах Курчатова
description : Легендарный научный руководитель советского атомного проекта академик Игорь Курчатов обожал шутить над коллегами. Госкорпорация «Росатом» вспомнила две истории.
site : 360tv.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-2.0850260446916997 url : https://www.f1news.ru/news/f1-141417.html
date : 2019-11-30 17:17:14
title : Грожан: Я рад, что завтра последняя гонка для этой машины
description : В квалификации в Абу-Даби машину Даниила Квята выпустили из боксов, когда по пит-лейн проезжал Роман Грожан. Французский гонщик избежал столкновения и заявил, что в этой сессии у него были гораздо более серьёзные проблемы, чем этот инцидент.
site : f1news.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.454907293294926 url : https://365news.biz/sport/237311-trudnyj-podrostok-kevin-de-brejne

-1.9248323883708622 url : https://newstula.ru/fn_504147.html
date : 2019-11-30 17:21:11
title : В Туле проходят Всероссийские соревнования по художественной гимнастике
description : На спортивные состязания съехались более 800 спортсменок из 30 субъектов РФ.
site : newstula.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-0.7313418916967517 url : https://sobesednik.ru/sport/20191130-pervyj-zimnij-uikend-i-vojna-dzyuby-so-spartakom-futbolnyj-gid-sobesednikru
date : 2019-11-30 17:21:11
title : Первый зимний уикенд и война Дзюбы со "Спартаком": футбольный гид Sobesednik.ru
description : Возвращение Гризманна на «Ванда Метрополитано», Дзюба против «Спартака»: дайджест главных матчей вых
site : sobesednik.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-0.9727197145578753 url : https://sobesednik.ru/sport/20191130-match-18-go-tura-rpl-perenesen-iz-za-zabolevaniya-desyati-igrokov-sochinskoj-komandy
date : 2019-11-30 17:21:11
title : Матч 18-го тура РПЛ перенесен из-за заболевания десяти игроков сочи

...

-1.871977603943698 url : https://www.tourprom.ru/news/44145/
date : 2019-11-30 13:04:41
title : На Лондонском мосту террорист с ножом напал на туристов
description : Популярное у туристов местов в центре Лондона – Лондонский мост – снова стало местом нападения, позднее признанного терактом
site : tourprom.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.8659304129929932 url : https://365news.biz/sport/237184-vezd-v-strany-evro-2020-dlya-bolelshhikov-iz-rossii-mozhet-byt-uproshhen.html
date : 2019-11-30 13:06:41
title : Въезд в страны ЕВРО-2020 для болельщиков из России может быть упрощен
description : ЕВРО-2020, Семён Галькевич. Въезд российских болельщиков в страны проведения чемпионата Европы по футболу 2020 года может быть упрощен на время турнира, заявил исполнительный директор УЕФА по организации мероприятий Мартин Каллен. Жеребьевка финальной части чемпионата Европы-2020 пройдет в субботу в Бухаресте. Сборные России, Бельгии и Дании узнают последнего соперника по группе B, им стане

-2.071493697145478 url : https://kazanreporter.ru/news/35303_strasnaa-davka-kriki-draka-kazanca-posle-koncerta-dva-casa-ne-mogli-zabrat-vesi
date : 2019-11-30 13:16:30
title : «Страшная давка. Крики. Драка». Казанца после концерта два часа не могли забрать вещи
description : По словам зрителей, виновны в инциденте организаторы концерта и администрация клуба.
site : kazanreporter.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.3269403785235612 url : https://fighttime.ru/news/item/19380-mikhail-koklyaev-nokauta-ne-bylo.html
date : 2019-11-30 13:18:39
title : Российский тяжелоатлет Михаил Кокляев, досрочно уступивший Александру Емельяненко в матче по правилам бокса, не собирается завершать бойцовскую карьеру, и рассчитывает взять у соперника реванш в поединке по правилам ММА.
description : Российский тяжелоатлет Михаил Кокляев, досрочно уступивший Александру Емельяненко в матче по правилам бокса, не собирается завершать бойцовскую карьеру, и рассчитывает взять у соперника реванш в поединке

-2.092474561518687 url : https://vk.com/@vsegda_sama-10-sovetov-krasoty2
date : 2019-11-30 10:49:15
title : 10 советов красоты:
description : Совет № 1 Каждое утро, натощак, съедайте ложку льняного семени, тщательно разжевывая и запивая стаканом воды. Затем выдержать 20 минут перед завтраком. Что это даст? Это средство замечательно очищает организм, главное – кишечник. Благодаря этому кожа станет более ровной и свежей. Будет наблюдаться небольшое, но здоровое похудение.
site : vk.com

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.9936811164278745 url : https://perm.aif.ru/incidents/v_permi_roditeli_byut_trevogu_iz-za_massovo_zabolevshih_uchenikov_gimnazii
date : 2019-11-30 10:49:24
title : В Перми родители бьют тревогу из-за массово заболевших учеников гимназии
description : Школьников тошнит прямо в коридорах
site : perm.aif.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.979586198473203 url : https://russian.rt.com/world/news/692161-volodin-ukraina-zheneva
date : 2019-11-30 10:46:31
titl

-1.3538348298719869 url : https://lenta.ru/news/2019/11/30/money/
date : 2019-11-30 10:50:00
title : Раскрыты гонорары Емельяненко и Кокляева за бой
description : Боец смешанного стиля (ММА) Александр Емельяненко получил миллион рублей за победу над штангистом Михаилом Кокляевым. Проигравший в бою Кокляев, в свою очередь, получил 500 тысяч рублей. Сертификаты спортсменам вручил представитель Федерации бокса России сразу после поединка.
site : lenta.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.3962208314184497 url : https://vologda-poisk.ru/news/sport/pavel-buchnevich-vnov-otlichilsya-v-nhl-video
date : 2019-11-30 10:50:26
title : Павел Бучневич вновь отличился в НХЛ (видео)
description : Воспитанник череповецкого хоккея Павел Бучневич открыл счет встречи в середине первого тайма.
site : vologda-poisk.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-2.0398698543193223 url : https://www.yesasia.ru/article/761561
date : 2019-11-30 10:49:23
title : Ким У Бин отдаст доходы от фанмитинга детям

-2.0891091417671483 url : https://vringe.com/video/129968-video-boya-aleksandr-emelyanenko-mikhail-koklyaev.htm
date : 2019-11-30 11:11:44
title : Видео боя: Александр Емельяненко — Михаил Кокляев
description : vRINGe.com предлагает вашему вниманию видеозапись боксёрского поединка между Александром Емельяненко и Михаилом Кокляевым, который состоялся 29 ноября 2018 года в Москве (РФ). Напоминаем, что записи многих других поединков вы всегда можете найти в нашем видеоархиве. Информация для правообладателей: данная видеозапись находится в публичном доступе на стороннем сайте, посему команда ресурса vRINGe не несёт ответственность за то, что она может содержать. По этой же причинe редакция ресурс...
site : vringe.com

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.1788303239046878 url : https://nation-news.ru/487953-vashington-odolel-tampu-v-nkhl-blagodarya-golam-ovechkina-kuznecova-i-orlova
date : 2019-11-30 11:11:43
title : "Вашингтон" одолел "Тампу" в НХЛ благодаря голам Овечкина, Кузнецова

In [36]:
### Попробывать обучить random_forest

In [37]:
%%time
word2id = {}
for item in dataset:
    item.words = list(get_doc_words(item))
    for word in item.words:
        if word not in word2id: word2id[word] = len(word2id)

Wall time: 1.81 s


In [38]:
id2word = dict( (v,k) for k,v in word2id.items())

In [39]:
len(word2id)

31067

In [40]:
def doc2vector(a_doc):
    res = [0] * len(word2id)
    if not hasattr(a_doc, 'words'): a_doc.words = get_doc_words(a_doc)
    for word in a_doc.words: 
        if word in word2id: #проверка для тестового сета
            res[word2id[word]] += 1 
    return res
        
doc2vector(sport_dataset[13])

[0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [41]:
from sklearn.ensemble import RandomForestClassifier

In [42]:
# classifier = RandomForestClassifier(n_estimators=3, max_depth=3)
classifier = RandomForestClassifier(n_estimators=3)

In [ ]:
X = []
y = []
for item in sport_dataset:
    X.append(doc2vector(item))
    y.append(1)
    
for item in not_sport_dataset:
    X.append(doc2vector(item))
    y.append(0)

In [ ]:
classifier.fit(X, y)

In [ ]:
for i, weight in sorted(enumerate(classifier.feature_importances_), key=lambda x: x[1], reverse=True):
    print(i, weight, id2word[i])

In [ ]:
classifier.predict_proba([doc2vector(not_sport_dataset[13])])

In [ ]:
for item in not_sport_dataset:
    vec = doc2vector(item)
    if classifier.predict([vec]):
        print(item)

In [ ]:
#### Тестовые данные
sport_test_dataset = [item for item in dataset_test if item.site in sport_sites]
not_sport_test_dataset = [item for item in dataset_test if item.site not in sport_sites]
print(len(sport_test_dataset), len(not_sport_test_dataset))

In [ ]:
X_test = []
y_test = []
for item in sport_test_dataset:
    X_test.append(doc2vector(item))
    y_test.append(1)
    
for item in not_sport_test_dataset:
    X_test.append(doc2vector(item))
    y_test.append(0)
    
y_test_pred = classifier.predict(X_test)

In [ ]:
import sklearn.metrics

In [ ]:
sklearn.metrics.precision_recall_fscore_support(y_test, y_test_pred)

In [ ]:
sklearn.metrics.confusion_matrix(y_test, y_test_pred)

In [ ]:
sklearn.metrics.confusion_matrix?

## Gensim

In [ ]:
### https://logic.pdmi.ras.ru/~sergey/slides/N16_DataFestLDA.pdf

In [ ]:
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from gensim.models import ldamulticore
from gensim.models import LdaModel

In [ ]:
# Create a corpus from a list of texts
common_dictionary = Dictionary(item.words for item in dataset)

In [ ]:
# common_dictionary.filter_extremes()

In [ ]:
def show_top_words(a_dictionary, a_top_n=100):
    for i, (id_word, freq) in enumerate(Counter(a_dictionary.dfs).most_common(a_top_n)):
        print(i, a_dictionary[id_word], freq, sep='\t')

In [ ]:
show_top_words(common_dictionary)

In [ ]:
# common_dictionary.filter_n_most_frequent(17)

In [ ]:
def filter_short_words(words):
    for word in words:
        if len(word) >= 3: yield word

In [ ]:
common_dictionary = Dictionary(filter_short_words(item.words) for item in dataset)

In [ ]:
show_top_words(common_dictionary)

In [ ]:
common_dictionary.filter_n_most_frequent(7)

In [ ]:
common_corpus = [common_dictionary.doc2bow(item.words) for item in dataset]

In [ ]:
# Train the model on the corpus.
# lda = ldamulticore.LdaMulticore(common_corpus, num_topics=10, workers=1)
lda = LdaModel(common_corpus, num_topics=100, id2word=common_dictionary)

In [ ]:
lda.show_topics(50)

In [ ]:
# Compute Perplexity
print('Perplexity: ', lda.log_perplexity(common_corpus))  # a measure of how good the model is. lower the better.

In [ ]:
doc_lda = lda[common_corpus]

In [ ]:
for i,item in itertools.islice(enumerate(doc_lda), 100):
    print(item)
    print(dataset[i])

In [ ]:
# # Build the bigram and trigram models
# bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
# trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# # Faster way to get a sentence clubbed as a trigram/bigram
# bigram_mod = gensim.models.phrases.Phraser(bigram)
# trigram_mod = gensim.models.phrases.Phraser(trigram)

# # See trigram example
# print(trigram_mod[bigram_mod[data_words[0]]])

In [ ]:
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, common_corpus, common_dictionary)
vis

In [ ]:
from gensim.models import word2vec

In [ ]:
fin = gzip.open('dataset_mai.jsonl.gz')
dataset_w2v = []
for line in itertools.islice(fin, 100000):
    data = json.loads(line.strip())
    dataset_w2v.append(Document(data))

In [ ]:
%%time
for item in dataset_w2v: item.words = list(get_doc_words(item))

In [ ]:
w2v = word2vec.Word2Vec([item.words for item in dataset_w2v], workers=4)

In [ ]:
w2v.most_similar('bmw') #

In [ ]:
w2v['king']

In [ ]:
from scipy.spatial.distance import cosine

In [ ]:
cosine(w2v['bmw'], w2v['bmw'])

In [ ]:
cosine(w2v['bmw'], w2v['mercedes'])

## ДЗ - реализовать поиск похожих документов по текстовым векторам и по word2vec векторам

In [ ]:
def get_word_match_most_similar_docs(a_doc, a_dataset, a_top_n=20):
    pass

def get_tf_idf_most_similar_doc(a_doc, dataset):
    # для каждого документа строится вектор размерности словаря (аналогично random forest) и добавляется idf
    # далее cosine между документами
    pass

def get_w2v_most_similar_doc(a_doc, dataset):
    # считается средний вектор по всем словам (можно при усреднении учитывать idf)
    pass

In [ ]:
doc_id = 13
for sim_func in [get_word_match_most_similar_docs, get_tf_idf_most_similar_doc, get_w2v_most_similar_doc]:
    sim_func(dataset[doc_id], dataset)